In [348]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using CUDA
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
	using Flux: @functor
	using ..Transformers: Abstract3DTensor, Container, epsilon, batchedmul, batched_triu!
	using Einsum
end
	

In [267]:
#Labeling: vielleicht ändern

begin
	labels = map(string, 1:10)
	startsym = "11"
	endsym = "12"
	unksym = "0"
	labels = [unksym, startsym, endsym, labels...]
	vocab = Vocabulary(labels, unksym)
end

Vocabulary{String}(13, unk=0)

In [268]:
#function for generate training datas 

#nichts ändern
sample_data() = (d = map(string, rand(1:10, 10)); (d,d))

sample_data (generic function with 1 method)

In [269]:
#nichts ändern

sample_data()

(["4", "10", "9", "1", "2", "3", "9", "8", "10", "6"], ["4", "10", "9", "1", "2", "3", "9", "8", "10", "6"])

In [270]:
#function for adding start & end symbol

#nichts ändern
preprocess(x) = [startsym, x..., endsym]

preprocess (generic function with 1 method)

In [271]:
#nichts ändern

begin
    @show sample_ex = preprocess.(sample_data())
    @show encoded_sample_ex = vocab(sample_ex[1]) #use Vocabulary to encode the training data
    end

sample_ex = preprocess.(sample_data()) = (["11", "9", "3", "8", "2", "10", "10", "1", "1", "3", "2", "12"], ["11", "9", "3", "8", "2", "10", "10", "1", "1", "3", "2", "12"])
encoded_sample_ex = vocab(sample_ex[1]) = [2, 12, 6, 11, 5, 13, 13, 4, 4, 6, 5, 3]


12-element Vector{Int64}:
  2
 12
  6
 11
  5
 13
 13
  4
  4
  6
  5
  3

In [272]:
#nichts ändern
sample = preprocess.(sample_data())

(["11", "5", "1", "9", "7", "10", "7", "8", "3", "3", "4", "12"], ["11", "5", "1", "9", "7", "10", "7", "8", "3", "3", "4", "12"])

In [273]:
#nichts ändern
encoded_sample = vocab(sample[1])

12-element Vector{Int64}:
  2
  8
  4
 12
 10
 13
 10
 11
  6
  6
  7
  3

In [274]:
#define a Word embedding layer which turn word index to word vector

#embeding vielleicht ändern
embed = Embed(512, length(vocab)) |> gpu

Embed(512)

In [275]:
#define a position embedding layer metioned above

#embeding vielleicht ändern
pe = PositionEmbedding(512) |> gpu

PositionEmbedding(512)

In [276]:
#wrapper for get embedding

#embeding vielleicht ändern
function embedding(x)
    we = embed(x, inv(sqrt(512)))
    e = we .+ pe(we)
    return e
  end

embedding (generic function with 1 method)

In [277]:
abstract type AbstractAttention end

In [278]:
struct LinearAttention{Q<:Dense, K<:Dense, V<:Dense, O<:Dense, DP<:Dropout} <: AbstractAttention
    head::Int
    future::Bool
    iqproj::Q
    ikproj::K
    ivproj::V
    oproj::O
    drop::DP
end

In [279]:
Flux.functor(mh::LinearAttention) = (mh.iqproj, mh.ikproj, mh.ivproj, mh.oproj), m -> LinearAttention(mh.head, mh.future, m..., mh.drop)


In [280]:
LinearAttention(head::Int,
                   is::Int,
                   hs::Int,
                   os::Int;
                   future::Bool=true, pdrop = 0.1) = LinearAttention(head,
                                                                        future,
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(hs*head, os),
                                                                        Dropout(pdrop),
                                                                        )


function Base.show(io::IO, mh::LinearAttention)
    hs = div(size(mh.iqproj.weight)[1], mh.head)
    is = size(mh.iqproj.weight)[end]
    os = size(mh.oproj.weight)[1]

    print(io, "LinearAttention(")
    print(io, "head=$(mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "$(is)=>$(os)")

    if Flux.istraining()
        print(io, ", dropout=$(mh.drop.p))")
    else
        print(io, ")")
    end
end

In [281]:
# of type float (to allow for integer inputs)
oftf(x, y) = oftype(float(x), y)
nelu(x, α=1) = ifelse(x ≥ 0, float(x)+1, @fastmath oftf(x, α) * (exp(x) - 1)+1)

nelu (generic function with 2 methods)

In [282]:
function splitHeads(x, batch, head, depth)
    x = reshape(x, (batch, -1, head, depth))
end    

splitHeads (generic function with 1 method)

In [347]:
function (mh::LinearAttention)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: Abstract3DTensor{T},
                         A2 <: Abstract3DTensor{T},
                         A3 <: Abstract3DTensor{T}}
qs = size(query)
ks = size(key)
vs = size(value)

#size(ipq) == (h, q_seq_len, batch)
ipq = @toNd mh.iqproj(query)
ipk = @toNd mh.ikproj(key)
ipv = @toNd mh.ivproj(value)

h = size(ipq, 1)
hs = div(h, mh.head)

ipq = nelu(splitHeads(ipq, qs[1], mh.head, hs))
ipk = nelu(splitHeads(ipk, qs[1], mh.head, hs))
ipv = nelu(splitHeads(ipv, qs[1], mh.head, hs))

@einsum k_v[m,d,e,h] := ipk[m,j,h,d]*ipv[m,j,h,e]

sum_1 = sum(ipk, axis=1) + 1e-8

@einsum z_1[m,l,h] := ipq[m,l,h,d]*sum_1[m,h,d]

z = 1/z_1

@einsum output[m,l,h,e] := ipq[m,l,h,d]*k_v[m,d,e,h]*z[m,l,h]

output = reshape(output, (qs[1], -1, mh.head*hs))

out = @toNd mh.oproj(output)

out

end

function (mh::LinearAttention)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: AbstractMatrix{T},
                         A2 <: AbstractMatrix{T},
                         A3 <: AbstractMatrix{T}}

# size(query) == (dims, seq_len)
ipq = mh.iqproj(query)
ipk = mh.ikproj(key)
ipv = mh.ivproj(value)

h = size(ipq)[1] #h == hs * head
hs = div(h, mh.head)

#size(hq) == (hs, seq_len, head)
hq = permutedims(reshape(ipq, hs, mh.head, :), [1, 3, 2])
hk = permutedims(reshape(ipk, hs, mh.head, :), [1, 3, 2])
hv = permutedims(reshape(ipv, hs, mh.head, :), [1, 3, 2])

@einsum k_v[m,d,e,h] := ipk[m,j,h,d]*ipv[m,j,h,e]

sum_1 = sum(ipk, axis=1) + 1e-8

@einsum z_1[m,l,h] := ipq[m,l,h,d]*sum_1[m,h,d]

z = 1/z_1

@einsum output[m,l,h,e] := ipq[m,l,h,d]*k_v[m,d,e,h]*z[m,l,h]

output = reshape(output, (qs[1], -1, mh.head*hs))

mh.oproj(output)
end



In [326]:
abstract type AbstractTransformer end

In [327]:
struct PwFFN{Di<:Dense, Do<:Dense}
    din::Di
    dout::Do
end

@functor PwFFN


"just a wrapper for two dense layer."
PwFFN(size::Int, h::Int, act = elu) = PwFFN(  #relu vielleicht auf elu + 1 oder nur elu ändern
    Dense(size, h, act),
    Dense(h, size)
)

function (pw::PwFFN)(x::AbstractMatrix)
  # size(x) == (dims, seq_len)
  pw.dout(pw.din(x))
end

function (pw::PwFFN)(x::A) where {T, N, A<:AbstractArray{T, N}}
  new_x = reshape(x, size(x, 1), :)
  y = pw(new_x)
  return reshape(y, Base.setindex(size(x), size(y, 1), 1))
end

In [328]:
struct Transformer1{MA<:LinearAttention, LA<:LayerNorm, P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
    mh::MA
    mhn::LA
    pw::P
    pwn::LP
    drop::DP
end

@functor Transformer1


"""
    Transformer(size::Int, head::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)
    Transformer(size::Int, head::Int, hs::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)  

Transformer layer.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
When `future` is `false`, the k-th token can't see the j-th tokens where j > k. `pdrop` is the dropout rate.
"""

########
#ganze Funktion von Transformer ändern
########
function Transformer1(size::Int, head::Int, ps::Int; future::Bool = true, act = elu, pdrop = 0.1)  #relu vielleicht wieder auf elu ändern
    rem(size, head) != 0 && error("size not divisible by head")
    Transformer1(size, head, div(size, head), ps;future=future, act=act, pdrop=pdrop)
end

Transformer1(size::Int, head::Int, hs::Int, ps::Int; future::Bool = true, act = elu, pdrop = 0.1) = Transformer1(
    LinearAttention(head, size, hs, size; future=future, pdrop=pdrop),
    LayerNorm(size),
    PwFFN(size, ps, act),
    LayerNorm(size),
    Dropout(pdrop),     #braucht man das? vielleicht nicht
)

function (t::Transformer1)(x::A, mask=nothing) where {T, N, A<:AbstractArray{T, N}}
    dropout = t.drop
    a = t.mh(x, x, x; mask=mask)
    a = dropout(a)
    res_a = x + a
    res_a = t.mhn(res_a)
    pwffn = t.pw(res_a)
    pwffn = dropout(pwffn)
    res_pwffn = res_a + pwffn
    res_pwffn = t.pwn(res_pwffn)
    res_pwffn
end

function Base.show(io::IO, t::Transformer1) #zum visualisieren --> nice to have
    hs = div(size(t.mh.iqproj.weight)[1], t.mh.head)
    h, ps = size(t.pw.dout.weight)

    print(io, "Transformer(")
    print(io, "head=$(t.mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "pwffn_size=$(ps), ")
    print(io, "size=$(h)")
    if Flux.istraining()
        print(io, ", dropout=$(t.drop.p))")
    else
        print(io, ")")
    end
end


In [329]:
###########
    #auch diese Funktion ändern
    ###############
    struct TransformerDecoder1{MA<:LinearAttention, LA<:LayerNorm,
        IMA<:LinearAttention, ILA<:LayerNorm,
        P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
mh::MA
mhn::LA
imh::IMA
imhn::ILA
pw::P
pwn::LP
drop::DP
end

@functor TransformerDecoder1

"""
TransformerDecoder(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
TransformerDecoder(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1)

TransformerDecoder layer. Decode the value from a Encoder.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
`pdrop` is the dropout rate.
"""
function TransformerDecoder1(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
rem(size, head) != 0 && error("size not divisible by head")
TransformerDecoder1(size, head, div(size, head), ps; act=act, pdrop=pdrop)
end

TransformerDecoder1(size::Int, head::Int, hs::Int, ps::Int; act = elu, pdrop = 0.1) = TransformerDecoder1(
LinearAttention(head, size, hs, size; future=false, pdrop=pdrop),
LayerNorm(size),
LinearAttention(head, size, hs, size; future=true, pdrop=pdrop), #future = true --> Unterschied zu oben??
LayerNorm(size),
PwFFN(size, ps, act),
LayerNorm(size),
Dropout(pdrop),
)

function (td::TransformerDecoder1)(x::AbstractArray{T,N}, m, mask=nothing) where {T,N}
dropout = td.drop
a = td.mh(x,x,x)
a = dropout(a)
res_a = x + a
res_a = td.mhn(res_a)

ia = td.imh(res_a, m, m, mask=mask)
ia = dropout(ia)
res_ia = res_a + ia
res_ia = td.imhn(res_ia)

pwffn = td.pw(res_ia)
pwffn = dropout(pwffn)
res_pwffn = res_ia + pwffn
res_pwffn = td.pwn(res_pwffn)
res_pwffn
end

function Base.show(io::IO, td::TransformerDecoder1)
hs = div(size(td.imh.iqproj.weight)[1], td.imh.head)
h, ps = size(td.pw.dout.weight)

print(io, "TransformerDecoder(")
print(io, "head=$(td.mh.head), ")
print(io, "head_size=$(hs), ")
print(io, "pwffn_size=$(ps), ")
print(io, "size=$(h)")
if Flux.istraining()
print(io, ", dropout=$(td.drop.p))")
else
print(io, ")")
end
end

In [330]:
#define 2 layer of transformer

#layer auf jeden Fall ändern
encode_t1 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [331]:
#define 2 layer of transformer decoder

#layer auf jeden Fall ändern
decode_t1 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [332]:
#define the layer to get the final output probabilities

#ÄNDERN
linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

Positionwise(Dense(512, 13), logsoftmax)

In [333]:
#neue Layer einfügen
function encoder_forward(x)
    e = embedding(x)
    t1 = encode_t1(e)
    return t1
  end

encoder_forward (generic function with 1 method)

In [334]:
#neue Layer einfügen
function decoder_forward(x, m)
    e = embedding(x)
    t1 = decode_t1(e, m)
    p = linear(t4)
    return p
  end

decoder_forward (generic function with 1 method)

In [335]:
#bleibt so
enc = encoder_forward(encoded_sample)

UndefVarError: UndefVarError: k_v not defined

In [336]:
#bleibt so
probs = decoder_forward(encoded_sample, enc)

UndefVarError: UndefVarError: enc not defined

In [337]:
#hier vielleicht etwas ändern --> herausfinden, was diese funktion macht
function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

smooth (generic function with 1 method)

In [338]:
Flux.@nograd smooth

In [339]:
#define loss function

#ÄNDERN
function loss(x, y)
    label = onehot(vocab, y) #turn the index to one-hot encoding
    label = smooth(label) #perform label smoothing
    enc = encoder_forward(x)
    probs = decoder_forward(y, enc)
    l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])  #erstmal so lassen
    return l
  end

loss (generic function with 1 method)

In [340]:
#collect all the parameters

#Layer neu einfügen
ps = params(embed, pe, encode_t1, decode_t1, linear)

Params([Float32[-0.2606126 0.6310123 … -0.6988065 -0.31930968; 1.8479936 0.7497308 … -0.7790108 1.4425807; … ; 0.81579834 1.3970042 … -0.46807522 0.95883757; -1.116791 -1.2467955 … 0.46665725 0.9255601], Float32[0.5403023 -0.41614684 … 0.4000682 0.9873536; 0.8218562 0.9364147 … 0.37902638 0.97646356; … ; 1.0 1.0 … 0.9943822 0.99437124; 0.0001 0.0002 … 0.10212166 0.10222114], Float32[0.044601362 -0.0039264224 … -0.073846765 0.04380427; -0.030616969 -0.010307705 … -0.0063672117 0.0038724567; … ; 0.015638055 -0.040032644 … 0.06287291 0.029638672; -0.056319125 -0.031322043 … -0.04193522 -0.07050639], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.058833025 -0.037865844 … -0.05559553 -0.07453927; -0.045890607 -0.0011352852 … 0.009711309 -0.035666943; … ; 0.059484445 0.036995806 … 0.075754136 0.024121715; 0.04377653 -0.013300689 … 0.05946855 0.046892773], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.

In [341]:
opt = ADAM(1e-4) #bleibt

ADAM(0.0001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [342]:
#define training loop

#ÄNDERN siehe Paper Kapitel 4.1
function train!()
    @info "start training"
    for i = 1:100
      data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
      x, y = preprocess.(data[1]), preprocess.(data[2])
      x, y = vocab(x), vocab(y) #encode the data
      x, y = todevice(x, y) #move to gpu
      grad = gradient(()->loss(x, y), ps)
      if i % 8 == 0
          l = loss(x, y)
          println("loss = $l")
      end
      update!(opt, ps, grad)
    end
  end

train! (generic function with 1 method)

In [343]:
train!()

DimensionMismatch: DimensionMismatch("new dimensions (512, -1, 8, 64) must be consistent with array size 196608")

In [344]:
#erst einmal so lassen, vielleicht einzelne Änderungen
function translate(x)
    ix = todevice(vocab(preprocess(x)))
    seq = [startsym]

    enc = encoder_forward(ix)

    len = length(ix)
    for i = 1:2len
        trg = todevice(vocab(seq))
        dec = decoder_forward(trg, enc)
        #move back to gpu due to argmax wrong result on CuArrays
        ntok = onecold(collect(dec), labels)
        push!(seq, ntok[end])
        ntok[end] == endsym && break
    end
  seq[2:end-1]
end

┌ Info: start training
└ @ Main /Users/johannes/Documents/GitHub/DM2022-LinearTransformers/test/LinearAttentionTransformer.ipynb:5


translate (generic function with 1 method)

In [345]:
translate(map(string, [5,5,6,6,1,12,3,4,6]))

UndefVarError: UndefVarError: k_v not defined